In [2]:
import csv
import numpy as np

with open('titanic-train.csv', 'rt') as f:
    data = list(csv.DictReader(f))
data[:1]

[{'Age': '22',
  'Cabin': '',
  'Embarked': 'S',
  'Fare': '7.25',
  'Name': 'Braund, Mr. Owen Harris',
  'Parch': '0',
  'PassengerId': '1',
  'Pclass': '3',
  'Sex': 'male',
  'SibSp': '1',
  'Survived': '0',
  'Ticket': 'A/5 21171'}]

In [3]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

_all_xs = [{k: v for k, v in row.items() if k != 'Survived'} for row in data]
_all_ys = np.array([int(row['Survived']) for row in data])

all_xs, all_ys = shuffle(_all_xs, _all_ys, random_state=0)
train_xs, valid_xs, train_ys, valid_ys = train_test_split(
    all_xs, all_ys, test_size=0.25, random_state=0)
print('{} items total, {:.1%} true'.format(len(all_xs), np.mean(all_ys)))

891 items total, 38.4% true


In [4]:
for x in all_xs:
    if x['Age']:
        x['Age'] = float(x['Age'])
    else:
        x.pop('Age')
    x['Fare'] = float(x['Fare'])
    x['SibSp'] = int(x['SibSp'])
    x['Parch'] = int(x['Parch'])

In [5]:
import warnings
# xgboost <= 0.6a2 shows a warning when used with scikit-learn 0.18+
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score


clf = XGBClassifier()
vec = DictVectorizer()
pipeline = make_pipeline(vec, clf)

def evaluate(_clf):
    scores = cross_val_score(_clf, all_xs, all_ys, scoring='accuracy', cv=10)
    print('Accuracy: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))
    _clf.fit(train_xs, train_ys)  # so that parts of the original pipeline are fitted

evaluate(pipeline)

Accuracy: 0.823 ± 0.071


In [6]:
booster = clf.booster()
original_feature_names = booster.feature_names
booster.feature_names = vec.get_feature_names()
print(booster.get_dump()[0])
# recover original feature names
booster.feature_names = original_feature_names

0:[Sex=female<-9.53674e-07] yes=1,no=2,missing=1
	1:[Age<13] yes=3,no=4,missing=4
		3:[SibSp<2] yes=7,no=8,missing=7
			7:leaf=0.145455
			8:leaf=-0.125
		4:[Fare<26.2687] yes=9,no=10,missing=9
			9:leaf=-0.151515
			10:leaf=-0.0727273
	2:[Pclass=3<-9.53674e-07] yes=5,no=6,missing=5
		5:[Fare<12.175] yes=11,no=12,missing=12
			11:leaf=0.05
			12:leaf=0.175194
		6:[Fare<24.8083] yes=13,no=14,missing=14
			13:leaf=0.0365591
			14:leaf=-0.152



In [7]:
from eli5 import show_weights
show_weights(clf, vec=vec)

Weight,Feature
0.4278,Sex=female
0.1949,Pclass=3
0.0665,Embarked=S
0.0510,Pclass=2
0.0420,SibSp
0.0417,Cabin=
0.0385,Embarked=C
0.0358,Ticket=1601
0.0331,Age
0.0323,Fare


In [8]:
from eli5 import show_prediction
show_prediction(clf, valid_xs[1], vec=vec, show_feature_values=True)

In [9]:
type(valid_xs[1])

dict

In [10]:
type(vec)

sklearn.feature_extraction.dict_vectorizer.DictVectorizer

In [11]:
import pandas as pd

data = pd.read_csv('titanic-train.csv')

In [14]:
data.loc[0]

PassengerId                          1
Survived                             0
Pclass                               3
Name           Braund, Mr. Owen Harris
Sex                               male
Age                                 22
SibSp                                1
Parch                                0
Ticket                       A/5 21171
Fare                              7.25
Cabin                              NaN
Embarked                             S
Name: 0, dtype: object

In [19]:
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [26]:
columns_list = list(data.columns)
print(columns_list)
columns_list.remove('Survived')
print(columns_list)

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [47]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

train_xs, valid_xs, train_ys, valid_ys = train_test_split(data[columns_list], data['Survived'], test_size=0.25, random_state=0)

In [48]:
import warnings
# xgboost <= 0.6a2 shows a warning when used with scikit-learn 0.18+
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score


clf = XGBClassifier()
#vec = DictVectorizer()
pipeline = make_pipeline(clf)
dummies_train = pd.get_dummies(train_xs[['Sex','Ticket','Cabin','Embarked']])
train_xs.drop(['Name','Sex','Ticket','Cabin','Embarked'],inplace = True, axis = 1)
train_xs = pd.concat([train_xs,dummies_train],axis = 1)
clf.fit(train_xs,train_ys)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [49]:
dummies_test = pd.get_dummies(valid_xs[['Sex','Ticket','Cabin','Embarked']])
valid_xs.drop(['Name','Sex','Ticket','Cabin','Embarked'],inplace = True,axis = 1)
valid_xs = pd.concat([valid_xs,dummies_test],axis = 1)

In [50]:
import eli5
eli5.show_prediction(clf,valid_xs.loc[1])

ValueError: feature_names mismatch: ['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male', 'Ticket_110152', 'Ticket_110413', 'Ticket_110465', 'Ticket_110564', 'Ticket_110813', 'Ticket_111240', 'Ticket_111320', 'Ticket_111361', 'Ticket_111369', 'Ticket_111426', 'Ticket_111427', 'Ticket_111428', 'Ticket_112050', 'Ticket_112052', 'Ticket_112059', 'Ticket_112277', 'Ticket_113028', 'Ticket_113050', 'Ticket_113051', 'Ticket_113055', 'Ticket_113056', 'Ticket_113059', 'Ticket_113501', 'Ticket_113503', 'Ticket_113505', 'Ticket_113510', 'Ticket_113572', 'Ticket_113760', 'Ticket_113767', 'Ticket_113773', 'Ticket_113776', 'Ticket_113781', 'Ticket_113783', 'Ticket_113784', 'Ticket_113786', 'Ticket_113787', 'Ticket_113788', 'Ticket_113789', 'Ticket_113792', 'Ticket_113796', 'Ticket_113798', 'Ticket_113803', 'Ticket_113804', 'Ticket_113806', 'Ticket_113807', 'Ticket_11668', 'Ticket_11751', 'Ticket_11752', 'Ticket_11753', 'Ticket_11755', 'Ticket_11765', 'Ticket_11767', 'T